In [ ]:
#######################################
Function: 
Make a data folder to store bamlist data.
#######################################

In [3]:
mkdir 00.data

mkdir: cannot create directory ‘00.data’: File exists


: 1

In [ ]:
#######################################
Function: 
Create a list that contains all of the BAM group names.

Output:
bams.lst(file that contains all of the BAMfiles)
#######################################

In [1]:
#write sample data
bamdir="/workspace/hramzr/DNAseq_mapping/alignment/05.add_read_group_IDs/"
f = open("00.data/bams.lst", "w")
for i in range(13):
    if i < 9:
        f.write(bamdir+"0"+str(i+1)+"_group.bam\n")
    else:
        f.write(bamdir+str(i+1)+"_group.bam\n")
f.close()

#### Generate vcf file with Freebayes

In [ ]:
#######################################
Function: 
Create a folder that stores log data and temporal data generated when running freebayes.
#######################################

In [2]:
mkdir -p 01.freebayes/log
mkdir -p 01.freebayes/tmp

In [ ]:
#######################################
Input:
nz_trevally.fasta(reference sequence)

Function: 
Index the reference sequence in order to generate VCF data from it with freebayes later on.

Output:
nz_trevally.fai(indexed reference sequence)
#######################################

In [3]:
module load samtools/1.9
#index ref and define REF
REF=/workspace/hramzr/nz_trevally.fasta
samtools faidx ${REF}

In [ ]:
#######################################
Input:
1. Reference sequence
2. BAM groups(by bamlist)

Function: 
Generate VCF data from the BAM groups as opposed to the reference.

Output:
animal_groups_to_ref.vcf(VCF file generated from 13 animal BAM files opposed to REF)
#######################################

In [4]:
export TMPDIR=01.freebayes/tmp


#define other variables
REFINDEX=${REF}.fai
BAM=00.data/bams.lst
PAR=/software/bioinformatics/freebayes-1.1.0/scripts
WKDIR=01.freebayes
module load freebayes/1.1.0
module load tabix/0.2.6
module load samtools

bsub << EOF
#!/bin/bash
freebayes-parallel <($PAR/fasta_generate_regions.py $REFINDEX 1000000) 60 \
-f $REF -L $BAM -C 5 -k --report-genotype-likelihood-max --min-mapping-quality 10 \
--genotype-qualities --use-mapping-quality --no-mnps --no-complex  > 01.freebayes/animal_groups_to_ref.vcf
EOF

Job <41427> is submitted to default queue <normal>.
